In [ ]:
!pip install ultralytics fiftyone opencv-python matplotlib


import cv2
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
import yaml
import fiftyone as fo
import fiftyone.zoo as foz
import random
import yaml
import importlib.resources as pkg_resources


In [ ]:
import os
import yaml
import importlib.resources as pkg_resources

# ------------------------------------------------------------------
# Helper functions: load dataset YAMLs, normalize names, resolve splits
# ------------------------------------------------------------------
def load_dataset_cfg(name):
    """Loads an Ultralytics dataset YAML from ultralytics/cfg/datasets."""
    path = pkg_resources.files("ultralytics") / "cfg" / "datasets" / name
    path = str(path)
    assert os.path.isfile(path), f"{path} not found"
    with open(path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg, path


def normalize_names(names):
    """Ensure names is a list of strings, regardless of dict/list form."""
    if isinstance(names, dict):
        return [names[i] for i in range(len(names))]
    elif isinstance(names, list):
        return names
    else:
        raise TypeError(f"Unsupported names type: {type(names)}")


def resolve_split(cfg, split_key):
    """
    Resolve a dataset split (train/val) from a dataset cfg to an absolute path.
    Handles relative paths from cfg['path'].
    """
    split = cfg.get(split_key, None)
    if split is None:
        raise KeyError(f"{split_key} not found in cfg")

    base_path = cfg.get("path", "")

    # If split is a list, just use the first element for now
    if isinstance(split, (list, tuple)):
        split = split[0]

    if base_path:
        split_path = os.path.join(base_path, split)
    else:
        split_path = split

    return os.path.abspath(split_path)


# ------------------------------------------------------------------
# Load COCO and HomeObjects-3K dataset configs
# ------------------------------------------------------------------
coco_cfg, coco_yaml_path = load_dataset_cfg("coco.yaml")
home_cfg, home_yaml_path = load_dataset_cfg("HomeObjects-3K.yaml")

print("COCO YAML path:", coco_yaml_path)
print("HomeObjects-3K YAML path:", home_yaml_path)

coco_names = normalize_names(coco_cfg["names"])
home_names = normalize_names(home_cfg["names"])

print(f"COCO num classes: {len(coco_names)}")
print(f"HomeObjects num classes: {len(home_names)}")


In [ ]:
# ---------------------------------------------------------------
# Build union names: COCO + HomeObjects
# Merge COCO "dining table" and HomeObjects "table" into "table"
# ---------------------------------------------------------------

# 1) Start with all COCO classes (COCO numeric IDs stay the same)
union_names = list(coco_names)

# Find the COCO index for "dining table" and rename it to "table"
try:
    dining_table_idx = coco_names.index("dining table")
    print("COCO 'dining table' class index:", dining_table_idx)
    union_names[dining_table_idx] = "table"
except ValueError:
    dining_table_idx = None
    print("[WARN] 'dining table' not found in COCO names! Will treat 'table' separately if present.")

# 2) Map HomeObjects classes into the union space
home_to_union = {}

for hid, hname in enumerate(home_names):
    target_name = hname

    # Simple normalization rules:
    if hname.lower() == "table":
        # Merge HomeObjects "table" into COCO's "table" class (if exists)
        if "table" in union_names:
            target_name = "table"
        else:
            target_name = "table"

    if hname.lower() == "sofa":
        # Map "sofa" to COCO "couch" if present
        if "couch" in union_names:
            target_name = "couch"

    # If this name already exists in union_names, reuse that ID
    if target_name in union_names:
        union_id = union_names.index(target_name)
    else:
        union_id = len(union_names)
        union_names.append(target_name)

    home_to_union[hid] = union_id

print("\nExample HomeObjects mapping:", list(home_to_union.items())[:10])

# 3) Create dictionary form for YAMLs
union_names_dict = {i: name for i, name in enumerate(union_names)}

print("\n[INFO] Final union class count (COCO+Home):", len(union_names))
print("[INFO] First 30 union classes:", union_names[:30])

# 4) Show where the splits resolve to (just info)
coco_train_images_guess = resolve_split(coco_cfg, "train")
coco_val_images_guess   = resolve_split(coco_cfg, "val")
home_train_images_guess = resolve_split(home_cfg, "train")
home_val_images_guess   = resolve_split(home_cfg, "val")

print("\nCOCO train images (guess):", coco_train_images_guess)
print("COCO val images (guess):  ", coco_val_images_guess)
print("Home train images (guess):", home_train_images_guess)
print("Home val images (guess):  ", home_val_images_guess)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
from pathlib import Path

# ---------------------------------------------------------------
# Unzip all Roboflow YOLOv8 datasets from Drive
# ---------------------------------------------------------------

# Folder in Drive containing your env zips
ENV_ZIP_DIR = Path("/content/drive/MyDrive/New-Yolo-Classes-Data/New_Yolo_Data")

# Where to extract them in the Colab filesystem
ENV_EXTRACT_ROOT = Path("/content/env_datasets")
ENV_EXTRACT_ROOT.mkdir(parents=True, exist_ok=True)

# Find all .zip files
env_zip_files = sorted([p for p in ENV_ZIP_DIR.iterdir() if p.suffix == ".zip"])

print("[INFO] Found env ZIP files:")
for z in env_zip_files:
    print("  •", z.name)

# Extract each zip to its own folder
extracted_datasets = []
for zip_path in env_zip_files:
    target_dir = ENV_EXTRACT_ROOT / zip_path.stem
    if not target_dir.exists():
        print(f"[INFO] Extracting {zip_path.name} → {target_dir}")
        target_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(target_dir)
    else:
        print(f"[INFO] Already extracted: {target_dir}")
    extracted_datasets.append(target_dir)

print("\n[INFO] extracted_datasets:")
for d in extracted_datasets:
    print("  •", d)


In [ ]:
import yaml
from pathlib import Path

# ---------------------------------------------------------------
# Load env datasets (Roboflow YOLOv8 exports) and add classes to union
# ---------------------------------------------------------------

# This must match where you unzipped the Roboflow zips:
ENV_ROOT = Path("/content/env_datasets")

print("[INFO] ENV_ROOT:", ENV_ROOT)
print("[INFO] ENV_ROOT exists:", ENV_ROOT.exists())
if ENV_ROOT.exists():
    print("[INFO] Contents of ENV_ROOT:")
    for p in ENV_ROOT.iterdir():
        print("   ", p, "(dir:" , p.is_dir(), ")")

env_sets = []      # list of {folder, train, val, names}
rf_to_union = {}   # folder name -> {local_id -> union_id}


def normalize(name: str) -> str:
    return name.lower().strip().replace("-", " ").replace("_", " ")

# Build lookup for current union names
union_lookup = {normalize(n): i for i, n in enumerate(union_names)}



# Find ALL data.yaml files under env_datasets, not just top-level
data_yamls = list(ENV_ROOT.rglob("data.yaml"))
print("\n[INFO] Found data.yaml files:")
for dy in data_yamls:
    print("   ", dy)

for data_yaml in data_yamls:
    ds_dir = data_yaml.parent  # directory that contains this data.yaml
    print(f"\n[INFO] Loading env dataset from {data_yaml}")

    with open(data_yaml, "r") as f:
        cfg = yaml.safe_load(f)





    names_list = cfg.get("names")
    if names_list is None:
      print(f"  [WARN] No 'names' in {data_yaml}, skipping.")
      continue

    # Ignore whatever 'train'/'val' say in data.yaml, and assume standard Roboflow layout:
    #   <dataset_root>/train/images
    #   <dataset_root>/valid/images
    train_root = (ds_dir / "train" / "images").resolve()
    val_root   = (ds_dir / "valid" / "images").resolve()

    train_root = str(train_root)
    val_root   = str(val_root)





    print(f"  train: {train_root}")
    print(f"  val:   {val_root}")
    print(f"  names: {names_list}")

    env_sets.append({
        "folder": ds_dir.name,
        "train": train_root,
        "val": val_root,
        "names": names_list,
    })

# Now integrate these names into the union space
print("\n[INFO] Integrating env datasets into union class space...")
for ds in env_sets:
    ds_map = {}
    for rf_id, rf_name in enumerate(ds["names"]):
        key = normalize(rf_name)
        if key in union_lookup:
            uid = union_lookup[key]
        else:
            uid = len(union_names)
            union_names.append(rf_name)
            union_lookup[key] = uid
        ds_map[rf_id] = uid
        print(f"[MAP] {ds['folder']} class {rf_id} ({rf_name}) → union {uid}")
    rf_to_union[ds["folder"]] = ds_map

print("\n[INFO] Final union class count (COCO+Home+env):", len(union_names))
print("[INFO] Number of env datasets loaded:", len(env_sets))


In [ ]:
# ---------------------------------------------------------------
# Integrate Roboflow env datasets into union class space
# ---------------------------------------------------------------
'''
def normalize(name: str) -> str:
    return name.lower().strip().replace("-", " ").replace("_", " ")

# Build lookup for current union names
union_lookup = {normalize(n): i for i, n in enumerate(union_names)}

rf_to_union = {}  # maps dataset folder -> { rf_id -> union_id }
'''

for ds in env_sets:
    ds_map = {}
    for rf_id, rf_name in enumerate(ds["names"]):
        key = normalize(rf_name)
        if key in union_lookup:
            uid = union_lookup[key]
        else:
            uid = len(union_names)
            union_names.append(rf_name)
            union_lookup[key] = uid
        ds_map[rf_id] = uid
        print(f"[MAP] {ds['folder']} class {rf_id} ({rf_name}) → union {uid}")
    rf_to_union[ds["folder"]] = ds_map

# Rebuild union_names_dict now that we've extended it
union_names_dict = {i: name for i, name in enumerate(union_names)}
print("\n[INFO] Final union class count (COCO+Home+env):", len(union_names))


In [ ]:
from ultralytics.data.utils import check_det_dataset
from glob import glob
import random, os

# 1) Download / locate COCO using Ultralytics
data_info = check_det_dataset("coco.yaml")
coco_train_root = os.path.abspath(data_info["train"])
coco_val_root   = os.path.abspath(data_info["val"])

print("Full COCO train root:", coco_train_root)
print("Full COCO val root:  ", coco_val_root)

# 2) Build list of ALL train images
if os.path.isdir(coco_train_root):
    # train is a directory of images
    all_coco_train_imgs = sorted(glob(os.path.join(coco_train_root, "*.jpg")))
else:
    # train is a .txt list – resolve any relative paths
    base = os.path.dirname(coco_train_root)
    all_coco_train_imgs = []
    with open(coco_train_root, "r") as f:
        for line in f:
            p = line.strip()
            if not p:
                continue
            if not os.path.isabs(p):
                p = os.path.join(base, p)
            if os.path.exists(p):
                all_coco_train_imgs.append(os.path.abspath(p))
            else:
                print("Skipping missing image:", p)

print("Total COCO train images found:", len(all_coco_train_imgs))

# 3) Sample a random subset of 5k
random.seed(0)
subset_size = min(7000, len(all_coco_train_imgs))
coco_train_subset = random.sample(all_coco_train_imgs, subset_size)

subset_txt = "coco_train5k.txt"
with open(subset_txt, "w") as f:
    for p in coco_train_subset:
        f.write(p + "\n")

coco_train_images = os.path.abspath(subset_txt)
coco_val_images   = coco_val_root  # or make a val subset similarly

print("Using COCO train subset file:", coco_train_images)
print("Using COCO val root:", coco_val_images)


In [ ]:

from ultralytics.data.utils import check_det_dataset
import os, yaml


# Ensure HomeObjects-3K is downloaded and get its real paths
home_info = check_det_dataset("HomeObjects-3K.yaml")  # downloads if needed
home_train_images = os.path.abspath(home_info["train"])
home_val_images   = os.path.abspath(home_info["val"])

print("Resolved Home train images:", home_train_images)
print("Resolved Home val images:  ", home_val_images)



In [ ]:
# ---------------------------------------------------------------
# Write union YAMLs: COCO + HomeObjects + Roboflow env datasets
# ---------------------------------------------------------------
union_names_dict = {i: name for i, name in enumerate(union_names)}

def write_yaml(path, data):
    with open(path, "w") as f:
        yaml.safe_dump(data, f, sort_keys=False)
    print("[INFO] Wrote", path)

# 1) Combined COCO + HomeObjects + env datasets for training
train_paths = [
    coco_train_images,
    home_train_images,
    home_train_images,  # oversample HomeObjects a bit
]

val_paths = [
    coco_val_images,
    home_val_images,
]

# Add env dataset paths, but only if the directory actually exists
for ds in env_sets:
    t = ds["train"]
    v = ds["val"]

    if t and os.path.isdir(t):
        train_paths.append(t)
    else:
        print(f"[WARN] Skipping env train path for {ds['folder']}: {t}")

    if v and os.path.isdir(v):
        val_paths.append(v)
    else:
        print(f"[WARN] Skipping env val path for {ds['folder']}: {v}")

print("\n[INFO] Final train paths:")
for p in train_paths:
    print("  ", p)

print("\n[INFO] Final val paths:")
for p in val_paths:
    print("  ", p)

coco_home_env_union = {
    "path": "",
    "train": train_paths,
    "val": val_paths,
    "names": union_names_dict,
}
write_yaml("coco_home_env_union.yaml", coco_home_env_union)
print("\n[INFO] Wrote coco_home_env_union.yaml")

In [ ]:
import glob, os

from ultralytics.data.utils import check_det_dataset

# 1) Ask Ultralytics where HomeObjects-3K really lives
home_info = check_det_dataset("HomeObjects-3K.yaml")
home_train_images = os.path.abspath(home_info["train"])
home_val_images   = os.path.abspath(home_info["val"])

print("Home train images:", home_train_images)
print("Home val images:  ", home_val_images)

# 2) Derive labels root by replacing 'images' with 'labels'
def images_to_labels_dir(images_dir):
    if "images" in images_dir:
        return images_dir.replace(os.sep + "images", os.sep + "labels")
    else:
        return os.path.join(os.path.dirname(images_dir), "labels")

home_labels_root_train = images_to_labels_dir(home_train_images)
home_labels_root_val   = images_to_labels_dir(home_val_images)

print("Home train labels root:", home_labels_root_train)
print("Home val labels root:  ", home_labels_root_val)

# 3) Collect all label files and list unique IDs used
label_files = glob.glob(os.path.join(home_labels_root_train, "**", "*.txt"), recursive=True)
label_files += glob.glob(os.path.join(home_labels_root_val, "**", "*.txt"), recursive=True)

used_ids = set()
for lf in label_files:
    with open(lf, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cid = int(float(parts[0]))
            used_ids.add(cid)

print("Class IDs used in HomeObjects labels:", sorted(used_ids))


In [ ]:
# This assumes you still have `home_to_union` dict and `home_cfg` in memory.
# If not, reload HomeObjects-3K.yaml and rebuild them as before.

def remap_homeobjects_labels_to_union_labels(home_labels_roots, home_to_union):
    """
    Remaps class IDs in YOLO label files under the given roots using home_to_union mapping.
    """
    import glob

    for root in home_labels_roots:
        if not os.path.isdir(root):
            print(f"[WARN] Labels root not found: {root}")
            continue

        txt_files = glob.glob(os.path.join(root, "**", "*.txt"), recursive=True)
        print(f"[INFO] Remapping {len(txt_files)} label files under {root}...")

        for lf in txt_files:
            with open(lf, "r") as f:
                lines = f.readlines()

            new_lines = []
            changed = False

            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue

                old_id = int(float(parts[0]))
                # Only remap HomeObjects IDs; leave anything else alone
                if old_id in home_to_union:
                    new_id = home_to_union[old_id]
                    parts[0] = str(new_id)
                    changed = True

                new_lines.append(" ".join(parts))

            if changed:
                with open(lf, "w") as f:
                    f.write("\n".join(new_lines) + "\n")

    print("[INFO] Finished remapping HomeObjects labels to union IDs.")


# Call it with the TRAIN + VAL label roots discovered above
home_labels_roots = [home_labels_root_train, home_labels_root_val]
remap_homeobjects_labels_to_union_labels(home_labels_roots, home_to_union)


In [ ]:
# ---------------------------------------------------------------
# Remap Roboflow env dataset labels into the union class ID space
# ---------------------------------------------------------------

import glob

def images_dir_to_labels_dir(images_dir: str) -> str:
    """
    Convert a typical YOLO/Roboflow images root to the corresponding
    labels root, e.g.:
      .../train/images  ->  .../train/labels
      .../valid/images  ->  .../valid/labels
    """
    from pathlib import Path

    p = Path(images_dir)
    parts = list(p.parts)
    if "images" in parts:
        idx = parts.index("images")
        parts[idx] = "labels"
        return str(Path(*parts))

    # Fallback: if "images" is not a separate path component
    return str(p.parent.parent / "labels" / p.parent.name)


def remap_env_labels_to_union(env_sets, rf_to_union):
    """
    For each Roboflow env dataset in env_sets, rewrite its YOLO label
    files so that class IDs use the global union IDs rather than the
    dataset-local IDs.
    """
    for ds in env_sets:
        folder_name = ds["folder"]           # e.g. "Kitchen_Env_1"
        mapping = rf_to_union.get(folder_name, None)
        if mapping is None:
            print(f"[WARN] No rf_to_union mapping for {folder_name}, skipping.")
            continue

        print(f"\n[INFO] Remapping env dataset '{folder_name}'")

        # derive labels roots from the train/val image roots
        image_roots = [ds["train"], ds["val"]]
        label_roots = [images_dir_to_labels_dir(r) for r in image_roots]

        for labels_root in label_roots:
            if not os.path.isdir(labels_root):
                print(f"  [WARN] Labels root not found: {labels_root}")
                continue

            txt_files = glob.glob(os.path.join(labels_root, "**", "*.txt"),
                                  recursive=True)
            print(f"  [INFO] Remapping {len(txt_files)} label files under {labels_root}...")

            for lf in txt_files:
                with open(lf, "r") as f:
                    lines = f.readlines()

                new_lines = []
                changed = False
                for line in lines:
                    parts = line.strip().split()
                    if not parts:
                        continue

                    # local class ID within this Roboflow dataset
                    local_id = int(float(parts[0]))
                    if local_id not in mapping:
                        # if it's some unexpected ID, leave the line as-is
                        print(f"    [WARN] class {local_id} not in mapping for {lf}, leaving as-is.")
                        new_lines.append(line.strip())
                        continue

                    union_id = mapping[local_id]
                    if str(union_id) != parts[0]:
                        parts[0] = str(union_id)
                        changed = True
                    new_lines.append(" ".join(parts))

                if changed:
                    with open(lf, "w") as f:
                        f.write("\n".join(new_lines) + "\n")

        print(f"[INFO] Finished remapping env dataset '{folder_name}'.")


# Actually run the remap for all env datasets
remap_env_labels_to_union(env_sets, rf_to_union)


In [ ]:
#find IDs for every class

for i, name in enumerate(union_names):
    print(i, name)


In [ ]:

KEEP_IDS = [0, 15, 16, 24, 25, 26, 28, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 85, 86, 88, 89, 90, 91]


In [ ]:
print("len(union_names):", len(union_names))
print("env_sets length:", len(env_sets))

for ds in env_sets:
    print("Env dataset:", ds["folder"])
    print("  names:", ds["names"])


In [ ]:

for i, hname in enumerate(home_names):
    uid = home_to_union[i]
    print(f"Home {i}: {hname} -> union {uid}: {union_names[uid]}")



In [ ]:
print("len(union_names):", len(union_names))
print("env_sets length:", len(env_sets))
for ds in env_sets:
    print(ds["folder"])
    print("  train:", ds["train"], "exists:", os.path.isdir(ds["train"]))
    print("  val:  ", ds["val"],   "exists:", os.path.isdir(ds["val"]))


In [ ]:

model = YOLO("yolov8n.pt")  # COCO-pretrained starting point

#calculate how many layers are in model (for freezing)
num_modules = len(list(model.model.model))


model.train(
    data="coco_home_env_union.yaml",
    epochs=50,          # fewer epochs, let early stopping do its thing
    imgsz=640,
    batch=16,
    lr0=5e-4,           # lower base LR to avoid big shifts
    lrf=0.01,
    freeze=num_modules-2,  # or freeze=num_modules-2 if you prefer your existing scheme
    patience=10,
    project="runs_union",
    name="yolov8n_union_coco5k_home_robo",
)




'''
#best so far
model.train(
    data="coco_home_union.yaml",
    epochs=50,          # fewer epochs
    #classes=KEEP_IDS,
    imgsz=640,
    batch=16,
    lr0=0.001,          # lower base LR (default is ~0.01)
    lrf=0.01,           # stronger decay, ends very small
    freeze=num_modules-2,          # freeze layers
    patience=10,
    project="runs_union",
    name="yolov8n_union_coco_home_gentle",
)
'''

In [ ]:
print(glob.glob("runs_union/**/weights/best*.pt", recursive=True))


In [ ]:
from ultralytics import YOLO

# Load the union-trained model

base_model = YOLO("yolov8n.pt")
union_model = YOLO("runs_union/yolov8n_union_coco5k_home/weights/best.pt")



def print_metrics(metrics, label=""):
    print(f"{label} MODEL PERFORMANCE")
    print(f"mAP@50:     {metrics.box.map50:.3f}")
    print(f"mAP@50-95:  {metrics.box.map:.3f}")
    print(f"Precision:  {metrics.box.mp:.3f}")
    print(f"Recall:     {metrics.box.mr:.3f}")
    print(f"F1 Score:   {metrics.box.f1[0]:.3f}")
    print("-" * 40)




In [ ]:
'''
# COCO (base model)
metrics_coco = base_model.val(data="coco_union.yaml", verbose=False)
print_metrics(metrics_coco, "BASE MODEL on COCO128")

# HomeObjects performance (base model)
metrics_home = base_model.val(data="homeobjects_union.yaml", verbose=False)
print_metrics(metrics_home, "BASE MODEL on HomeObjects-3K")
'''


# COCO (fine-tuned model)
metrics_coco = union_model.val(data="coco_union.yaml", verbose=False)
print_metrics(metrics_coco, "UNION MODEL on COCO128")

# HomeObjects performance (fine-tuned model)
metrics_home = union_model.val(data="homeobjects_union.yaml", verbose=False)
print_metrics(metrics_home, "UNION MODEL on HomeObjects-3K")


# Save to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Example for Ultralytics YOLOv8
import shutil

# Path to the trained weights in Colab's temporary storage
source_best_weights = 'runs_union/yolov8n_union_coco5k_home_robo3/weights/best.pt'

# Destination path in Google Drive
destination_folder = '/content/drive/MyDrive/BestYoloModel'

# Create the destination folder if it doesn't exist
import os
os.makedirs(destination_folder, exist_ok=True)

# Copy the best weights
shutil.copy(source_best_weights, os.path.join(destination_folder, 'YOLOModel6_withRobo.pt'))

print(f"YOLO model weights saved to {destination_folder}")



# Predict Video

In [ ]:

def test_yolo_video(model, video_path):
    """
    Test YOLOv8 model on a video.

    Args:
        model: The loaded YOLOv8 model.
        video_path: The path to the input video file.
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform inference on the frame
        results = model(frame, classes=KEEP_IDS)#, conf=0.2)
        res = results[0]

        # Display the frame with detections
        plt.imshow(res.plot())
        plt.axis('off')
        plt.show()

    cap.release()
    print("Video processing finished.")

In [ ]:
#test_yolo_video(YOLO("runs_union/yolov8n_union_coco_home_gentle/weights/best.pt"), "/content/215475_small.mp4")

test_yolo_video(YOLO("runs_union/yolov8n_union_coco5k_home_robo3/weights/best.pt"), "/content/215475_small.mp4")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def test_yolo_video_with_positions(model, video_path, keep_ids=None, show=True):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0

    if not cap.isOpened():
        print(f"[ERROR] Could not open video: {video_path}")
        return

    # Try to get class names from the model once
    model_names = None
    try:
        # works for Ultralytics YOLO models
        model_names = model.model.names
    except Exception:
        pass

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLO on this frame
        if keep_ids is not None:
            results = model(frame, classes=keep_ids)
        else:
            results = model(frame)

        res = results[0]

        # Fallback: if we didn’t get names from the model, try from the result
        names = model_names or getattr(res, "names", None)

        # Get boxes in xywh format (center x,y,width,height)
        if res.boxes is not None and len(res.boxes) > 0:
            boxes_xywh = res.boxes.xywh.cpu().numpy()
            clss       = res.boxes.cls.cpu().numpy()
            confs      = res.boxes.conf.cpu().numpy()
        else:
            boxes_xywh = []
            clss       = []
            confs      = []

        print(f"\n[FRAME {frame_idx}] detections: {len(boxes_xywh)}")

        for (x, y, w, h), c, conf in zip(boxes_xywh, clss, confs):
            cls_id = int(c)

            # Resolve class name from names if available
            cls_name = None
            if names is not None:
                # names can be list or dict
                if isinstance(names, dict):
                    cls_name = names.get(cls_id, None)
                else:  # list/tuple
                    if 0 <= cls_id < len(names):
                        cls_name = names[cls_id]

            if cls_name is None:
                cls_name = f"id_{cls_id}"

            print(
                f"  {cls_name:20s} "
                f"center=({x:.1f}, {y:.1f}), "
                f"w={w:.1f}, h={h:.1f}, conf={conf:.2f}"
            )

        # Optionally show the plotted frame with boxes
        if show:
            plotted = res.plot()  # draws boxes + labels
            # res.plot() returns BGR when using OpenCV under the hood;
            # if colors look off, flip channels:
            plt.imshow(plotted[..., ::-1])
            plt.axis('off')
            plt.show()

        frame_idx += 1

    cap.release()


In [ ]:
from ultralytics import YOLO

baseModel = YOLO("yolov8n.pt")
modelTest = YOLO("/content/YOLOModel6_withRobo.pt")
test_yolo_video_with_positions(modelTest, "/content/IMG_6406.MOV", keep_ids=KEEP_IDS, show=True)
